# LanceDB

>[LanceDB](https://lancedb.com/) is an open-source database for vector-search built with persistent storage, which greatly simplifies retrevial, filtering and management of embeddings. Fully open source.

This notebook shows how to use functionality related to the `LanceDB` vector database based on the Lance data format.

In [ ]:
! pip install tantivy

In [ ]:
! pip install -U langchain-openai langchain-community

In [ ]:
! pip install lancedb

We want to use OpenAIEmbeddings so we have to get the OpenAI API Key. 

In [1]:
import getpass
import os

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

In [2]:
! rm -rf /tmp/lancedb

In [3]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import LanceDB
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter

loader = TextLoader("../../how_to/state_of_the_union.txt")
documents = loader.load()

documents = CharacterTextSplitter().split_documents(documents)
embeddings = OpenAIEmbeddings()

##### For LanceDB cloud, you can invoke the vector store as follows :


```python
db_url = "db://lang_test" # url of db you created
api_key = "xxxxx" # your API key
region="us-east-1-dev"  # your selected region

vector_store = LanceDB(
    uri=db_url,
    api_key=api_key,
    region=region,
    embedding=embeddings,
    table_name='langchain_test'
    )
```

You can also add `region`, `api_key`, `uri` to `from_documents()` classmethod


In [4]:
from lancedb.rerankers import LinearCombinationReranker

reranker = LinearCombinationReranker(weight=0.3)

docsearch = LanceDB.from_documents(documents, embeddings, reranker=reranker)
query = "What did the president say about Ketanji Brown Jackson"

In [31]:
docs = docsearch.similarity_search_with_relevance_scores(query)
print("relevance score - ", docs[0][1])
print("text- ", docs[0][0].page_content[:1000])

relevance score -  0.7066475030191711
text-  They were responding to a 9-1-1 call when a man shot and killed them with a stolen gun. 

Officer Mora was 27 years old. 

Officer Rivera was 22. 

Both Dominican Americans who’d grown up on the same streets they later chose to patrol as police officers. 

I spoke with their families and told them that we are forever in debt for their sacrifice, and we will carry on their mission to restore the trust and safety every community deserves. 

I’ve worked on these issues a long time. 

I know what works: Investing in crime prevention and community police officers who’ll walk the beat, who’ll know the neighborhood, and who can restore trust and safety. 

So let’s not abandon our streets. Or choose between safety and equal justice. 

Let’s come together to protect our communities, restore trust, and hold law enforcement accountable. 

That’s why the Justice Department required body cameras, banned chokeholds, and restricted no-knock warrants for it

In [33]:
docs = docsearch.similarity_search_with_score(query="Headaches", query_type="hybrid")
print("distance - ", docs[0][1])
print("text- ", docs[0][0].page_content[:1000])

distance -  0.30000001192092896
text-  My administration is providing assistance with job training and housing, and now helping lower-income veterans get VA care debt-free.  

Our troops in Iraq and Afghanistan faced many dangers. 

One was stationed at bases and breathing in toxic smoke from “burn pits” that incinerated wastes of war—medical and hazard material, jet fuel, and more. 

When they came home, many of the world’s fittest and best trained warriors were never the same. 

Headaches. Numbness. Dizziness. 

A cancer that would put them in a flag-draped coffin. 

I know. 

One of those soldiers was my son Major Beau Biden. 

We don’t know for sure if a burn pit was the cause of his brain cancer, or the diseases of so many of our troops. 

But I’m committed to finding out everything we can. 

Committed to military families like Danielle Robinson from Ohio. 

The widow of Sergeant First Class Heath Robinson.  

He was born a soldier. Army National Guard. Combat medic in Kosovo and 

In [8]:
print("reranker : ", docsearch._reranker)

reranker :  <lancedb.rerankers.linear_combination.LinearCombinationReranker object at 0x107ef1130>


Additionaly, to explore the table you can load it into a df or save it in a csv file: 
```python
tbl = docsearch.get_table()
print("tbl:", tbl)
pd_df = tbl.to_pandas()
# pd_df.to_csv("docsearch.csv", index=False)

# you can also create a new vector store object using an older connection object:
vector_store = LanceDB(connection=tbl, embedding=embeddings)
```

In [15]:
docs = docsearch.similarity_search(
    query=query, filter={"metadata.source": "../../how_to/state_of_the_union.txt"}
)

print("metadata :", docs[0].metadata)

# or you can directly supply SQL string filters :

print("\nSQL filtering :\n")
docs = docsearch.similarity_search(query=query, filter="text LIKE '%Officer Rivera%'")
print(docs[0].page_content)

metadata : {'source': '../../how_to/state_of_the_union.txt'}

SQL filtering :

They were responding to a 9-1-1 call when a man shot and killed them with a stolen gun. 

Officer Mora was 27 years old. 

Officer Rivera was 22. 

Both Dominican Americans who’d grown up on the same streets they later chose to patrol as police officers. 

I spoke with their families and told them that we are forever in debt for their sacrifice, and we will carry on their mission to restore the trust and safety every community deserves. 

I’ve worked on these issues a long time. 

I know what works: Investing in crime prevention and community police officers who’ll walk the beat, who’ll know the neighborhood, and who can restore trust and safety. 

So let’s not abandon our streets. Or choose between safety and equal justice. 

Let’s come together to protect our communities, restore trust, and hold law enforcement accountable. 

That’s why the Justice Department required body cameras, banned chokeholds, and r

## Adding images 

In [ ]:
! pip install -U langchain-experimental

In [ ]:
! pip install open_clip_torch torch

In [16]:
! rm -rf '/tmp/multimmodal_lance'

In [17]:
from langchain_experimental.open_clip import OpenCLIPEmbeddings

In [18]:
import os

import requests

# List of image URLs to download
image_urls = [
    "https://github.com/raghavdixit99/assets/assets/34462078/abf47cc4-d979-4aaa-83be-53a2115bf318",
    "https://github.com/raghavdixit99/assets/assets/34462078/93be928e-522b-4e37-889d-d4efd54b2112",
]

texts = ["bird", "dragon"]

# Directory to save images
dir_name = "./photos/"

# Create directory if it doesn't exist
os.makedirs(dir_name, exist_ok=True)

image_uris = []
# Download and save each image
for i, url in enumerate(image_urls, start=1):
    response = requests.get(url)
    path = os.path.join(dir_name, f"image{i}.jpg")
    image_uris.append(path)
    with open(path, "wb") as f:
        f.write(response.content)

In [21]:
from langchain_community.vectorstores import LanceDB

vec_store = LanceDB(
    table_name="multimodal_test",
    embedding=OpenCLIPEmbeddings(),
)

In [22]:
vec_store.add_images(uris=image_uris)

['b673620b-01f0-42ca-a92e-d033bb92c0a6',
 '99c3a5b0-b577-417a-8177-92f4a655dbfb']

In [23]:
vec_store.add_texts(texts)

['f7adde5d-a4a3-402b-9e73-088b230722c3',
 'cbed59da-0aec-4bff-8820-9e59d81a2140']

In [24]:
img_embed = vec_store._embedding.embed_query("bird")

In [25]:
vec_store.similarity_search_by_vector(img_embed)[0]

Document(page_content='bird', metadata={'id': 'f7adde5d-a4a3-402b-9e73-088b230722c3'})

In [26]:
vec_store._table

LanceTable(connection=LanceDBConnection(/tmp/lancedb), name="multimodal_test")